# Initialisation

Import libraries, load setting and load data.

In [1]:
import numpy as np
import json
import datetime
from sklearn import linear_model, kernel_ridge

import utils


path_base = "../"

# Load settings
with open(path_base+"SETTINGS.json", 'r') as fid:
    settings = json.load(fid)
    print("Settings loaded.")
    
# Load store info
store_info = utils.loadStoreInfo(path_base+settings["STORE_INFO_PATH"])
print("Store information: {} lines loaded".format(len(store_info)))

# Load training data
(input_data, output_sales, output_customers) = utils.loadTrainingData(path_base+settings["TRAIN_DATA_PATH"])
print("Training data: {} lines loaded".format(len(input_data)))

print(store_info[0])

Settings loaded.
Store information: 1115 lines loaded
Training data: 1017209 lines loaded
[1, 'c', 'a', 1270, 9, 2008, 0, -1, -1, '']


# 1. Creation of the features

Here we define the functions that generates vectors of $\mathbb{R}^{D}$ from the entries.

In [42]:
# Long version for non-kernel models


# Define dimension of the feature space
D = 15


def generateFeatureVector(entry):
    ################################## Local functions
    
    # Local function to convert store asortment
    def ConvertAsortment(assortment):
        if assortment == "a":
            return 1
        elif assortment == "b":
            return 2
        else:
            return 3
        
    def GetCompetTime(entry):
        if store_info[entry[0]-1][4] == -1:
            return 0   # No entry available
        else:
            competition_start = datetime.date(
                store_info[entry[0]-1][5],  # Year
                store_info[entry[0]-1][4],  # Month
                1)                          # Day (no entry available)
            
            delta = entry[2] - competition_start
            return delta.days
        
    def GetStateHoliday(entry):
        if entry[5] == "0":
            return 0
        elif entry[5] == "a": # Public Holiday
            return 1
        elif entry[5] == "b": # Easter
            return 2
        else:
            return 3
            
    def GetSchoolHoliday(entry):
        return int(entry[6])
    
    ################################# End of local functions
    
    # Compute the number of days since competition
    nb_days_since_competition = GetCompetTime(entry)
    if nb_days_since_competition > 0:
        corrected_nb_days_comp = nb_days_since_competition
    else:
        corrected_nb_days_comp = 0
    
    # Compute distance to competitor
    if nb_days_since_competition>0:
        dist_comp = store_info[entry[0]-1][3]
    else:
        dist_comp = 0
    
    
    vector = np.array([
        # Store information
        dist_comp,                                          # Distance to competitor
        dist_comp**2,                                       # Squared distance to competitor
        ConvertAsortment(store_info[entry[0]-1][2]),        # Assortment
        ConvertAsortment(store_info[entry[0]-1][2])**2,     # Squared assortment
        1.0/(1+corrected_nb_days_comp),                     # Number of days since competition
        corrected_nb_days_comp,                             
        corrected_nb_days_comp**2,
        # Day information
        entry[1],                    # Day of the week
        1 if (entry[1]==6 or entry[1]==7) else 0,          # is week-end ?
        entry[2].day,                # Day of month
        entry[2].month,              # Month
        entry[2].year,               # Year
        entry[4],                    # Promo
        GetStateHoliday(entry),      # State Holiday
        GetSchoolHoliday(entry)
    ])
    
    return vector

In [7]:
# Short version for kernel ridge regression

# Define dimension of the feature space
D = 8

def generateFeatureVector(entry):
    ################################## Local functions
    
    # Local function to convert store asortment
    def ConvertAsortment(assortment):
        if assortment == "a":
            return 1
        elif assortment == "b":
            return 2
        else:
            return 3
        
    def GetCompetTime(entry):
        if store_info[entry[0]-1][4] == -1:
            return 0   # No entry available
        else:
            competition_start = datetime.date(
                store_info[entry[0]-1][5],  # Year
                store_info[entry[0]-1][4],  # Month
                1)                          # Day (no entry available)
            
            delta = entry[2] - competition_start
            return delta.days
            
    
    ################################# End of local functions
    
    # Compute the number of days since competition
    nb_days_since_competition = GetCompetTime(entry)
    
    # Compute distance to competitor
    if nb_days_since_competition>0:
        dist_comp = store_info[entry[0]-1][3]
    else:
        dist_comp = 0
    
    
    vector = np.array([
        # Store information
        dist_comp,                                          # Distance to competitor
        ConvertAsortment(store_info[entry[0]-1][2]),        # Assortment
        nb_days_since_competition,                          # Number of days since competition
        # Day information
        entry[1],                    # Day of the week
        entry[2].day,                # Day of month
        entry[2].month,              # Month
        entry[2].year,               # Year
        entry[4]                     # Promo
    ])
    
    return vector

# 2. Creation of the training set / test set


In [43]:
pourcentage_training_set = 80

# Split the entries into training data and test data
(tr_input, tr_label_sales, tr_label_customers,
     te_input, te_label_sales, te_label_customers) = utils.separateTrainingSet(
        input_data,
        output_sales, 
        output_customers,
        pourcentage_training_set)

# Convert the labels into Numpy arrays
training_labels = np.array(tr_label_sales)
test_labels = np.array(te_label_sales)

N = training_labels.shape[0]
N_test = test_labels.shape[0]

# Generate features of data set and test set
training_set = np.zeros((N,D))
for i in range(N):
    training_set[i,:] = generateFeatureVector(tr_input[i])
    
test_set = np.zeros((N_test, D))
for i in range(N_test):
    test_set[i,:] = generateFeatureVector(te_input[i])
    

# Summary
print("Dimensions of the training set: ({},{})".format(
        training_set.shape[0], training_set.shape[1]))
print("Dimensions of the test set: ({},{})".format(
        test_set.shape[0], test_set.shape[1]))

print(training_set[0,:])
print(tr_input[0])
print(training_set[:,4].min())
print(training_set[:,4].max())


Dimensions of the training set: (813767,15)
Dimensions of the test set: (203442,15)
[  0.00000000e+00   0.00000000e+00   3.00000000e+00   9.00000000e+00
   1.00000000e+00   0.00000000e+00   0.00000000e+00   3.00000000e+00
   0.00000000e+00   1.80000000e+01   1.20000000e+01   2.01300000e+03
   1.00000000e+00   0.00000000e+00   0.00000000e+00]
[736, 3, datetime.date(2013, 12, 18), 1, 1, '0', '0']
2.36882624659e-05
1.0


# 3. Model training

### Linear regression

First, we test a simple linear regression, we expect it to perform poorly because of outliers.

In [44]:
print("RMSPE avec la moyenne: {}".format(
    utils.compute_RMSPE(test_labels, training_labels.mean()*np.ones((test_labels.shape[0])))))

# Linear regression without normalization
model_linreg = linear_model.LinearRegression()
model_linreg.fit(training_set, training_labels)

print("Coefficients: ")
print(model_linreg.coef_)

y_hat = model_linreg.predict(test_set)
print(y_hat[:20])
print(test_labels[:20])


print("Linear regression without normalisation: RMSPE = {}".format(
    utils.compute_RMSPE(test_labels, y_hat)))

# Linear regression with normalization
model_linreg_n = linear_model.LinearRegression(fit_intercept=True,normalize=True,n_jobs=2)
model_linreg_n.fit(training_set, training_labels)

y_hat_n = model_linreg_n.predict(test_set)

print("Linear regression with normalisation: RMSPE = {}".format(
    utils.compute_RMSPE(test_labels, y_hat_n)))


RMSPE avec la moyenne: 0.48349623001200076
Coefficients: 
[ -3.64218266e-02   6.24535079e-07   1.14251107e+04  -2.78088520e+03
  -2.01008540e+02  -3.85006197e-02   1.79447925e-06  -5.02013018e+02
  -1.03465877e+03   6.26537843e+00   7.62853783e+01   1.74139304e+02
   2.25028678e+03  -3.41919632e+03   2.37698305e+02]
[ 2999.82778956  3257.46127648  2512.16691931  1757.08013354  6000.85288808
  7408.33648057  5387.95486051  9024.19489092  8351.47830709  7280.79982349
  2906.34743992  7138.20944301  3048.45604364  2840.14821236  6940.99146685
  7183.05726976  5292.97157156  7167.81484012  2647.32198469  7096.80879373]
[ 4027  6343     0     0  4823  8693  4655  5786 11801  9398     0  4761
  6361     0  5513  6122  5686  5784     0  5249]
Linear regression without normalisation: RMSPE = 0.48160179600808795
Linear regression with normalisation: RMSPE = 0.4816017962726558


The order of magnitude of the RMSPE with linear regression is 0.45, slightly better than just predicting the mean value of the sales in the training set. 

### Ridge regression

Then, a more complicated model: a ridge regression to handle outliers. 

In [45]:
model_ridgereg = linear_model.RidgeCV(
    alphas = [0.0001,0.001,0.003,0.01,0.1,1.0,10],
    normalize=True)

model_ridgereg.fit(training_set, training_labels)

print("Hyperparameter selected: alpha = {}".format(
    model_ridgereg.alpha_))

y_hat_rr = model_ridgereg.predict(test_set)

print("Ridge regression with normalisation: RMSPE = {}".format(
    utils.compute_RMSPE(test_labels, y_hat_rr)))

Hyperparameter selected: alpha = 0.0001
Ridge regression with normalisation: RMSPE = 0.48088346888899614


### Kernel Ridge Regression
Then a regression with different kernels to better handle non-linearities

In [46]:
#model_kridge = kernel_ridge.KernelRidge(alpha=0.001,kernel='rbf',gamma=0.01)
#model_kridge = kernel_ridge.KernelRidge(alpha=0.001,kernel='rbf',gamma=0.0001)

#model_kridge = kernel_ridge.KernelRidge(alpha=0.001,kernel='rbf',gamma=0.001)

#model_kridge = kernel_ridge.KernelRidge(alpha=0.001,kernel='linear',degree=3, coef0=1)

model_kridge = kernel_ridge.KernelRidge(alpha=0.001,kernel="sigmoid", gamma=0.0002, coef0=0.00001)

# Normalize dataset
data_means = training_set.mean(axis=0)
data_std = training_set.std(axis=0)

n_training_set = np.zeros(training_set.shape)
n_test_set = np.zeros(test_set.shape)

for i in range(training_set.shape[0]):
    n_training_set[i,:] = (training_set[i,:]-data_means)/data_std
    
for i in range(test_set.shape[0]):
    n_test_set[i,:] = (test_set[i,:]-data_means)/data_std

"""
del training_set
del test_set
del input_data
del te_input
del tr_input
"""

# Train model
model_kridge.fit(n_training_set[:1200,:], training_labels[:1200])

# Evaluate model
y_hat_kridge = model_kridge.predict(n_test_set)

print("Kernel ridge regression with normalisation: RMSPE = {}".format(
    utils.compute_RMSPE(test_labels, y_hat_kridge)))


Kernel ridge regression with normalisation: RMSPE = 0.43870803934482755
